<a href="https://colab.research.google.com/github/thotasriharsha/GENERATIVE-AI2/blob/main/2303A52159_WEEK_7_ASS_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below.
Train, test, save (.h5) and deploy the model to diagnose diabatic disease using the Keras deep
learning library
2. (1 ponto) Calculate training and testing accuracy, build confusion matrix, also calculate recall,
precision and F1-score.
3. (1 ponto) Build the application by loading the saved ANN model.
Tabela 1: ANN Architecture
Layer Neurons Activation Function
Hidden Layer - 1 8 relu
Hidden Layer - 2 16 relu
Hidden Layer - 3 20 relu
Hidden Layer - 4 10 relu
Tabela 2: Training Parameters
epochs batch size error metric Optimizer
150 64 accuracy adadelta



In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset from the URL
url = "/content/Housing (3).csv"
column_names = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
    'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]

# Load the dataset into a pandas DataFrame
df = pd.read_csv(url, header=None, names=column_names)

# Preview the dataset
print(df.head())


                                 Pregnancies   Glucose BloodPressure  \
price    area bedrooms bathrooms     stories  mainroad     guestroom   
13300000 7420 4        2                   3       yes            no   
12250000 8960 4        4                   4       yes            no   
         9960 3        2                   2       yes            no   
12215000 7500 4        2                   2       yes            no   

                                 SkinThickness          Insulin  \
price    area bedrooms bathrooms      basement  hotwaterheating   
13300000 7420 4        2                    no               no   
12250000 8960 4        4                    no               no   
         9960 3        2                   yes               no   
12215000 7500 4        2                   yes               no   

                                              BMI DiabetesPedigreeFunction  \
price    area bedrooms bathrooms  airconditioning                  parking   
13300000

In [4]:
# Features (X) and Target (y)
X = df.drop(columns='Outcome')
y = df['Outcome']

# Convert columns to numeric, handling errors
for column in X.columns:
    try:
        # Apply pd.to_numeric to each element individually
        X[column] = X[column].apply(pd.to_numeric, errors='coerce')
    except ValueError:
        print(f"Column '{column}' contains non-numeric values. Handling or removing them is recommended.")
        # Here, you can choose to handle the non-numeric values in a way that suits your data:
        # 1. Remove rows with non-numeric values:
        # X = X[pd.to_numeric(X[column], errors='coerce').notna()]
        # 2. Replace non-numeric values with a specific value (e.g., mean, median):
        # X[column] = pd.to_numeric(X[column], errors='coerce').fillna(X[column].mean())
        # 3. Implement other custom handling logic based on your understanding of the data.

    # Impute missing values after conversion
    # Choose an appropriate strategy (mean, median, etc.)
    X[column] = X[column].fillna(X[column].mean())

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check the shape of the splits
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (436, 8), X_test shape: (110, 8)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [5]:
# Create the model
model = Sequential()

# Adding the hidden layers
model.add(Dense(8, input_dim=X_train.shape[1], activation='relu'))  # First hidden layer
model.add(Dense(16, activation='relu'))  # Second hidden layer
model.add(Dense(20, activation='relu'))  # Third hidden layer
model.add(Dense(10, activation='relu'))  # Fourth hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer (sigmoid for binary classification)

# Compile the model
model.compile(optimizer=Adadelta(), loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model architecture
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 8)                   │              72 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │             144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │             340 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             210 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 777 (3.04 KB)

 Trainable params: 777 (3.04 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Features (X) and Target (y)
X = df.drop(columns='Outcome')
y = df['Outcome']

# Convert columns to numeric, handling errors
for column in X.columns:
    try:
        # Apply pd.to_numeric to each element individually
        X[column] = X[column].apply(pd.to_numeric, errors='coerce')
    except ValueError:
        print(f"Column '{column}' contains non-numeric values. Handling or removing them is recommended.")
        # Here, you can choose to handle the non-numeric values in a way that suits your data:
        # 1. Remove rows with non-numeric values:
        # X = X[pd.to_numeric(X[column], errors='coerce').notna()]
        # 2. Replace non-numeric values with a specific value (e.g., mean, median):
        # X[column] = pd.to_numeric(X[column], errors='coerce').fillna(X[column].mean())
        # 3. Implement other custom handling logic based on your understanding of the data.

    # Impute missing values after conversion
    # Choose an appropriate strategy (mean, median, etc.)
    X[column] = X[column].fillna(X[column].mean())

# Convert 'Outcome' column to numeric (int or float)
y = pd.to_numeric(y, errors='coerce')
# If there are non-numeric values in 'Outcome', you might need to handle them:
# For example, replace non-numeric with the most frequent value:
# y = y.fillna(y.mode()[0]).astype(int) # Replace with mode and convert to int

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check the shape of the splits
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (436, 8), X_test shape: (110, 8)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [7]:
# Features (X) and Target (y)
X = df.drop(columns='Outcome')
y = df['Outcome']

# Convert columns to numeric, handling errors
for column in X.columns:
    try:
        # Apply pd.to_numeric to each element individually
        X[column] = X[column].apply(pd.to_numeric, errors='coerce')
    except ValueError:
        print(f"Column '{column}' contains non-numeric values. Handling or removing them is recommended.")
        # Here, you can choose to handle the non-numeric values in a way that suits your data:
        # 1. Remove rows with non-numeric values:
        # X = X[pd.to_numeric(X[column], errors='coerce').notna()]
        # 2. Replace non-numeric values with a specific value (e.g., mean, median):
        # X[column] = pd.to_numeric(X[column], errors='coerce').fillna(X[column].mean())
        # 3. Implement other custom handling logic based on your understanding of the data.

    # Impute missing values after conversion
    # Choose an appropriate strategy (mean, median, etc.)
    X[column] = X[column].fillna(X[column].mean())

# Convert 'Outcome' column to numeric (int or float)
y = pd.to_numeric(y, errors='coerce')
# If there are non-numeric values in 'Outcome', you might need to handle them:
# For example, replace non-numeric with the most frequent value:
#y = y.fillna(y.mode()[0]).astype(int) # Replace with mode and convert to int

#Handle NaN values in 'Outcome' by replacing them with the most frequent value
# Check if y.mode() is empty before accessing element 0
if not y.mode().empty:
    y = y.fillna(y.mode()[0]) # Replace NaN with mode
else:
    # Handle the case where there is no mode (e.g., all unique values)
    # You might want to fill with a different value or strategy
    y = y.fillna(y.mean())  # Example: Fill with the mean

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Check the shape of the splits
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (436, 8), X_test shape: (110, 8)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.11/dist-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [8]:
# Save the trained model
model.save('diabetes_model.h5')
print("Model saved as 'diabetes_model.h5'")


Model saved as 'diabetes_model.h5'


In [9]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('diabetes_model.h5')

# Make a prediction on a new example (let's use the first row from the test set)
new_data = X_test[0].reshape(1, -1)  # Reshape the data to match the input shape
prediction = (loaded_model.predict(new_data) > 0.5).astype(int)

# Print the prediction result
print(f'Prediction: {"Positive" if prediction == 1 else "Negative"}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Prediction: Negative
